In [ ]:
# This script inserts data into Accelerometer_Generated 
# It finds the recording_ids in Accelerometer_Raw that don't have an associated raw_id in Accelerometer_Generated
# if re-doing baby data, make sure to delete previous generated data

In [ ]:
import pandas as pd
import math
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine

In [ ]:
m_window = 10000 # in ms
feature_window = 2000 # in ms

In [ ]:
# Converts timestamp to milliseconds
def dateTimeToMS(dateTime):
    millisec = dateTime.timestamp() * 1000
    return millisec

In [ ]:
def generateData(rawData):
    fullData = pd.DataFrame()
    # calculate m
    for row in rawData.itertuples():
        i = row.Index
        rawData.at[i, 'millisec'] = dateTimeToMS(row.timestamp)

    # subtract the mean from the data within a time window centered around the current sample
    for row in rawData.itertuples():
        alxList = [row.arm_left_x]
        alyList = [row.arm_left_y]
        alzList = [row.arm_left_z]
        arxList = [row.arm_right_x]
        aryList = [row.arm_right_y]
        arzList = [row.arm_right_z]
        llxList = [row.leg_left_x]
        llyList = [row.leg_left_y]
        llzList = [row.leg_left_z]
        lrxList = [row.leg_right_x]
        lryList = [row.leg_right_y]
        lrzList = [row.leg_right_z]
        listNames = ['arm_left_x','arm_left_y','arm_left_z','arm_right_x','arm_right_y','arm_right_z','leg_left_x','leg_left_y',
                    'leg_left_z','leg_right_x','leg_right_y','leg_right_z'] 
        columnList = [alxList,alyList,alzList,arxList,aryList,arzList,llxList,llyList,
                      llzList,lrxList,lryList,lrzList] 

        centerTime = row.millisec
        rowIndex = row.Index
        laterNotDone = True
        beforeNotDone = True
        numBefore = 1
        numAfter = 1

        while beforeNotDone and rowIndex - numBefore >= 0:
            if centerTime - rawData.loc[rowIndex-numBefore].millisec <= m_window/2 :
                nameIndex = 0
                for clist in columnList:
                    column = listNames[nameIndex]
                    clist.append(rawData.loc[rowIndex-numBefore][column])
                    nameIndex += 1
                numBefore += 1
            else:
                beforeNotDone = False
                numBefore = 1

        while laterNotDone and rowIndex + numAfter < len(rawData):
            if rawData.loc[rowIndex+numAfter].millisec - centerTime <= m_window/2 :

                nameIndex = 0
                for clist in columnList:
                    column = listNames[nameIndex]
                    clist.append(rawData.loc[rowIndex+numAfter][column])
                    nameIndex += 1
                numAfter += 1
            else:
                laterNotDone = False
                numAfter = 1

        alxc = row.arm_left_x - np.mean(alxList)
        alyc = row.arm_left_y - np.mean(alyList)
        alzc = row.arm_left_z - np.mean(alzList)
        arxc = row.arm_right_x - np.mean(arxList)
        aryc = row.arm_right_y - np.mean(aryList)
        arzc = row.arm_right_z - np.mean(arzList)
        llxc = row.leg_left_x - np.mean(llxList)
        llyc = row.leg_left_y - np.mean(llyList)
        llzc = row.leg_left_z - np.mean(llzList)
        lrxc = row.leg_right_x - np.mean(lrxList)
        lryc = row.leg_right_y - np.mean(lryList)
        lrzc = row.leg_right_z - np.mean(lrzList)    

        mla = math.sqrt(alxc**2 + alyc**2 + alzc**2)
        mra = math.sqrt(arxc**2 + aryc**2 + arzc**2)
        mll = math.sqrt(llxc**2 + llyc**2 + llzc**2)
        mrl = math.sqrt(lrxc**2 + lryc**2 + lrzc**2)

        fullData = fullData.append({'raw_id':row.id,
                                    'timestamp':row.millisec,
                                    'left_arm_x_calibrated':alxc,'left_arm_y_calibrated':alyc,'left_arm_z_calibrated':alzc,
                                    'right_arm_x_calibrated':arxc,'right_arm_y_calibrated':aryc,'right_arm_z_calibrated':arzc,
                                    'left_leg_x_calibrated':llxc,'left_leg_y_calibrated':llyc,'left_leg_z_calibrated':llzc,
                                    'right_leg_x_calibrated':lrxc,'right_leg_y_calibrated':lryc,'right_leg_z_calibrated':lrzc,
                                    'm_left_arm': mla, 'm_right_arm':mra, 'm_left_leg':mll, 'm_right_leg':mrl}, 
                                     ignore_index=True)

    # calculate max and prod for each row
    for row in fullData.itertuples():  
        rowIndex = row.Index
        max_mla_mra = max(row.m_left_arm, row.m_right_arm)
        max_mll_mrl = max(row.m_left_leg, row.m_right_leg)
        max_mla_mll = max(row.m_left_arm, row.m_left_leg)
        max_mra_mrl = max(row.m_right_arm, row.m_right_leg)
        max_mra_mla_mrl_mll = max(row.m_right_arm, row.m_left_arm, row.m_right_leg, row.m_left_leg)
        prod_mla_mra = row.m_left_arm * row.m_right_arm
        prod_mll_mrl = row.m_left_leg * row.m_right_leg
        prod_mla_mll = row.m_left_arm * row.m_left_leg
        prod_mra_mrl = row.m_right_arm * row.m_right_leg
        prod_mra_mla_mrl_mll = row.m_right_arm * row.m_left_arm * row.m_right_leg * row.m_left_leg


        fullData.at[rowIndex, 'max_mla_mra'] = max_mla_mra
        fullData.at[rowIndex, 'max_mll_mrl'] = max_mll_mrl
        fullData.at[rowIndex, 'max_mla_mll'] = max_mla_mll
        fullData.at[rowIndex, 'max_mra_mrl'] = max_mra_mrl
        fullData.at[rowIndex, 'max_mra_mla_mrl_mll'] = max_mra_mla_mrl_mll
        fullData.at[rowIndex, 'prod_mla_mra'] = prod_mla_mra
        fullData.at[rowIndex, 'prod_mll_mrl'] = prod_mll_mrl
        fullData.at[rowIndex, 'prod_mla_mll'] = prod_mla_mll
        fullData.at[rowIndex, 'prod_mra_mrl'] = prod_mra_mrl
        fullData.at[rowIndex, 'prod_mra_mla_mrl_mll'] = prod_mra_mla_mrl_mll

    # calculate the mean, max, min, standard deviation, and z-score 
    # from data within a time window centered around the current sample
    for row in fullData.itertuples(): 
        rowIndex = row.Index

        m_left_arm = [row.m_left_arm]
        m_right_arm = [row.m_right_arm]
        m_left_leg = [row.m_left_leg]
        m_right_leg = [row.m_right_leg]

        max_mla_mra = [row.max_mla_mra]
        max_mll_mrl = [row.max_mll_mrl]
        max_mla_mll = [row.max_mla_mll]
        max_mra_mrl = [row.max_mra_mrl]
        max_mra_mla_mrl_mll = [row.max_mra_mla_mrl_mll]

        prod_mla_mra = [row.prod_mla_mra]
        prod_mll_mrl = [row.prod_mll_mrl]
        prod_mla_mll = [row.prod_mla_mll]
        prod_mra_mrl = [row.prod_mra_mrl]
        prod_mra_mla_mrl_mll = [row.prod_mra_mla_mrl_mll]

        listNames = ['m_left_arm','m_right_arm','m_left_leg','m_right_leg',
                     'max_mla_mra','max_mll_mrl','max_mla_mll','max_mra_mrl','max_mra_mla_mrl_mll',
                     'prod_mla_mra','prod_mll_mrl','prod_mla_mll','prod_mra_mrl','prod_mra_mla_mrl_mll'] 
        featureList = [m_left_arm,m_right_arm,m_left_leg,m_right_leg,
                       max_mla_mra,max_mll_mrl,max_mla_mll,max_mra_mrl,max_mra_mla_mrl_mll,
                       prod_mla_mra,prod_mll_mrl,prod_mla_mll,prod_mra_mrl,prod_mra_mla_mrl_mll] 

        centerTime = row.timestamp
        laterNotDone = True
        beforeNotDone = True
        numBefore = 1
        numAfter = 1

        while beforeNotDone and rowIndex - numBefore  >= 0:
            if centerTime - fullData.loc[rowIndex-numBefore].timestamp <= feature_window/2 :
                nameIndex = 0
                for ftList in featureList:
                    ft = listNames[nameIndex]
                    ftList.append(fullData.loc[rowIndex-numBefore][ft])
                    nameIndex += 1
                numBefore += 1
            else:
                beforeNotDone = False
                numBefore = 1

        while laterNotDone and rowIndex + numAfter < len(fullData):
            if fullData.loc[rowIndex+numAfter].timestamp - centerTime <= feature_window/2 :
                nameIndex = 0
                for ftList in featureList:
                    ft = listNames[nameIndex]
                    ftList.append(fullData.loc[rowIndex+numAfter][ft])
                    nameIndex += 1
                numAfter += 1
            else:
                laterNotDone = False
                numAfter = 1

        nameIndex = 0        
        for ftList in featureList:
            ft = listNames[nameIndex]
            std = np.std(ftList)
            mean = np.mean(ftList)
            meanft = 'time_mean_of_' + ft
            maxft = 'time_max_of_' + ft
            minft = 'time_min_of_' + ft
            stdft = 'time_std_of_' + ft
            zft = 'time_z_score_of_' + ft
            fullData.at[rowIndex, meanft] = mean
            fullData.at[rowIndex, maxft] = max(ftList)
            fullData.at[rowIndex, minft] = min(ftList)
            fullData.at[rowIndex, stdft] = std
            if(std != 0.) :
                fullData.at[rowIndex, zft] = np.divide(fullData.loc[rowIndex][ft] - mean, std)
            
            nameIndex += 1

        # calculate pearson correlation coefficient for (left arm, right arm) and (right leg, left leg)
        mean_l_arm = np.mean(m_left_arm)
        mean_r_arm = np.mean(m_right_arm)
        mean_l_leg = np.mean(m_left_leg)
        mean_r_leg = np.mean(m_right_leg)
        prod_arm_diff_sum = 0
        prod_leg_diff_sum = 0
        right_arm_diff_sq_sum = 0
        left_arm_diff_sq_sum = 0
        right_leg_diff_sq_sum = 0
        left_leg_diff_sq_sum = 0

        for i in range(len(m_left_arm)):
            left_arm_diff = m_left_arm[i] - mean_l_arm
            right_arm_diff = m_right_arm[i] - mean_r_arm
            prod_arm_diff_sum += right_arm_diff * left_arm_diff
            right_arm_diff_sq_sum += right_arm_diff ** 2
            left_arm_diff_sq_sum += left_arm_diff ** 2  

            left_leg_diff = m_left_leg[i] - mean_l_leg
            right_leg_diff = m_right_leg[i] - mean_r_leg
            prod_leg_diff_sum += right_leg_diff * left_leg_diff
            right_leg_diff_sq_sum += right_leg_diff ** 2
            left_leg_diff_sq_sum += left_leg_diff ** 2  
        
        # Make sure it's not diving by zero
        if prod_arm_diff_sum * right_arm_diff_sq_sum * left_arm_diff_sq_sum != 0. :
            arm_correlation_coefficient = prod_arm_diff_sum / (math.sqrt(right_arm_diff_sq_sum) * math.sqrt(left_arm_diff_sq_sum))
            leg_correlation_coefficient = prod_leg_diff_sum / (math.sqrt(right_leg_diff_sq_sum) * math.sqrt(left_leg_diff_sq_sum))
            fullData.at[rowIndex, 'pearson_time_mla_mra'] = arm_correlation_coefficient
            fullData.at[rowIndex, 'pearson_time_mll_mrl'] = leg_correlation_coefficient
    fullData = fullData.drop('timestamp', axis = 1)
    return fullData

In [ ]:
import config # this is a file on my local computer that has my database login information
db_engine = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'.format(config.login['username'], config.login['password'], config.login['host'], '3306', config.login['database']))
connection = db_engine.connect()

In [ ]:
# query the recording ids that don't already have data generated
idList = []

not_done_ids = ("SELECT DISTINCT recording_id "
                "FROM Accelerometer_Raw "
                "WHERE id "
                "NOT IN (SELECT raw_id FROM Accelerometer_Generated)")
ids = connection.execute(not_done_ids)
for row in ids:
    idList.append(row[0])

In [ ]:
for recording_id in idList:
    queryData = ("SELECT * "
                 "FROM Accelerometer_Raw "
                 "WHERE recording_id = %s "
                 "ORDER BY timestamp" % recording_id)
    rawData = pd.read_sql(queryData, connection)
    
    # call the function to generate the data
    generatedData = generateData(rawData)
    
    # add generated data to the Accelerometer_Generated table
    generatedData.to_sql('Accelerometer_Generated', con=connection, index=False, if_exists='append')
    
connection.close()